In [ ]:
#!pip install tensorflow==2.5.0

In [1]:
import tensorflow as tf
import numpy as np
from tensorflow import keras
from tensorflow.keras.optimizers import RMSprop

In [ ]:
class myCallback(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs={}):
    if(logs.get('accuracy') is not None and logs.get('accuracy') >= 0.8): # Experiment with changing this value
      print("\nReached 80% accuracy so cancelling training!")
      self.model.stop_training = True

## **MISSION: 데이터 전처리**
이 데이터로 모델을 학습시키기 위해선 각종 전처리가 필요합니다.

TFDS 패키지를 이용한 것이 아니므로 ImageDataGenerator를 이용하여 전처리를 진행할 것입니다.

<br>

0-255 사이의 pixel 값을 0-1 사이의 float 값으로 Normalize를 실시하고,
Train / Validation 의 비율을 8:2로 설정하겠습니다.

<br>

설정한 데이터를 flow_from_directory를 이용하여 Resize, batch_size, class_mode를 지정하겠습니다.
데이터의 크기는 150x150 으로 Resize를 실시할 것이며,
batch_size는 20,
class_mode는 categorical로 지정하겠습니다.
여기서 categorical로 지정하면 label 정보가 one-hot encoding으로 설정됩니다.
<br>

ImageDataGenerator에 Augmentation을 적용 -> 속도가 느려질순 있음


In [ ]:
TRAINING_DIR = "tmp/rps/"

training_datagen = ImageDataGenerator(
  rescale=1. / 255,
  validation_split=0.2,
  rotation_range=5,
  width_shift_range=0.2,
  height_shift_range=0.2,
  horizontal_flip=True
)

# 카테고리로 설정해서 진행할 경우는 이렇게
# 애초에 dir가 나누어져 있으면 각각 디렉토리 따로 설정한다. validatio_split 없이
train_generator = training_datagen.flow_from_directory(
      TRAINING_DIR, 
      target_size=(150, 150), 
      batch_size=20, 
      class_mode='categorical', 
      subset='training', #validation 비율 따라서 나눠짐
    )

validation_generator = training_datagen.flow_from_directory(
      TRAINING_DIR, 
      target_size=(150, 150), 
      batch_size=20, 
      class_mode='categorical',
      subset='validation',
      )


In [ ]:
# 이렇게 데이터를 데이터를 전처리하는 경우도 있었음
def preprocess(features):
  
    img = features['image']
    label = features['label']
    img = tf.image.resize(img, [224, 224])
    img = tf.cast(img, tf.float32)
    img = img / 255.0

    return img, label

train_dataset = dataset.map(preprocess).batch(32)

## **MISSION: 네트워크 학습**
## **예시 답안**

아까 flow_from_directory에서 Categorical로 class_mode를 지정하였으니 loss 함수로는 Catogoricalentropy를 사용합니다.
혹시나 sparse로 지정하였다면, SparseCategoricalCrossentropy를 사용하여 compile 합니다.
Optimizer의 경우에는 SGD, Adam, RMSprop, ... 등 아무거나 선택하셔도 무방합니다.

<br>

학습은 15 epoch을 실시하였고, 더 줄이거나 늘려도 문제 없습니다.
validation accuracy가 80% 이상 나오도록 위의 네트워크 구조 및 학습 epoch 수를 변경하시면 좋습니다.

In [ ]:
model = tf.keras.models.Sequential([
        tf.keras.layers.Conv2D(16, (3, 3), activation='relu', input_shape=(150, 150, 3)),
        tf.keras.layers.MaxPooling2D(2, 2),
        tf.keras.layers.Conv2D(32, (3, 3), activation='relu'),
        tf.keras.layers.MaxPooling2D(2, 2),
        tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
        tf.keras.layers.MaxPooling2D(2, 2),
        tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
        tf.keras.layers.MaxPooling2D(2, 2),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(256, activation='relu'),
        tf.keras.layers.Dense(256, activation='relu'),
        tf.keras.layers.Dense(3, activation='softmax') #분류를 몇개로 하나에 따라서 값 바뀜
    ])

model.compile(optimizer=tf.keras.optimizers.RMSprop(),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# model.compile(optimizer=RMSprop(lr=0.001),
#               loss='binary_crossentropy',
#               metrics = ['accuracy'])

model.fit(train_generator, epochs=15, validation_data=validation_generator)
model.save("rps.h5")

## 위에는 3개 분류라서 categorical
## binary 예제는 아래

In [ ]:
!wget --no-check-certificate \
    https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip \
    -O /tmp/cats_and_dogs_filtered.zip
  
import os
import zipfile
import tensorflow as tf
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.preprocessing.image import ImageDataGenerator

local_zip = '/tmp/cats_and_dogs_filtered.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('/tmp')
zip_ref.close()

base_dir = '/tmp/cats_and_dogs_filtered'
train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'validation')

train_cats_dir = os.path.join(train_dir, 'cats') # Directory with our training cat pictures
train_dogs_dir = os.path.join(train_dir, 'dogs') # Directory with our training dog pictures
validation_cats_dir = os.path.join(validation_dir, 'cats') # Directory with our validation cat pictures
validation_dogs_dir = os.path.join(validation_dir, 'dogs')# Directory with our validation dog pictures

#데이터 확인
train_cat_fnames = os.listdir( train_cats_dir )
train_dog_fnames = os.listdir( train_dogs_dir )
print(train_cat_fnames[:10])
print(train_dog_fnames[:10])
print('total training cat images :', len(os.listdir(train_cats_dir ) ))
print('total training dog images :', len(os.listdir(train_dogs_dir ) ))
print('total validation cat images :', len(os.listdir( validation_cats_dir ) ))
print('total validation dog images :', len(os.listdir( validation_dogs_dir ) ))

model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, (3,3), activation='relu', input_shape=(150, 150, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

model.compile(loss='binary_crossentropy',
              optimizer=RMSprop(lr=1e-4),
              metrics=['accuracy'])

# Train용 에다가만 augmentation 부여!
train_datagen = ImageDataGenerator(
      rescale=1./255,
      rotation_range=40,
      width_shift_range=0.2,
      height_shift_range=0.2,
      shear_range=0.2,
      zoom_range=0.2,
      horizontal_flip=True,
      fill_mode='nearest')

test_datagen = ImageDataGenerator(rescale=1./255)

# Flow training images in batches of 20 using train_datagen generator
train_generator = train_datagen.flow_from_directory(
        train_dir,  # This is the source directory for training images
        target_size=(150, 150),  # All images will be resized to 150x150
        batch_size=20,
        # Since we use binary_crossentropy loss, we need binary labels
        class_mode='binary')

# Flow validation images in batches of 20 using test_datagen generator
validation_generator = test_datagen.flow_from_directory(
        validation_dir,
        target_size=(150, 150),
        batch_size=20,
        class_mode='binary')

history = model.fit(
      train_generator,
      steps_per_epoch=100,  # 2000 images = batch_size * steps
      epochs=100,
      validation_data=validation_generator,
      validation_steps=50,  # 1000 images = batch_size * steps
      verbose=2)

# 데이터 학습정도 그래프로 확인

In [ ]:
#-----------------------------------------------------------
# Retrieve a list of list results on training and test data
# sets for each training epoch
#-----------------------------------------------------------
acc      = history.history[     'accuracy' ]
val_acc  = history.history[ 'val_accuracy' ]
loss     = history.history[    'loss' ]
val_loss = history.history['val_loss' ]

epochs   = range(len(acc)) # Get number of epochs

#------------------------------------------------
# Plot training and validation accuracy per epoch
#------------------------------------------------
plt.plot  ( epochs,     acc )
plt.plot  ( epochs, val_acc )
plt.title ('Training and validation accuracy')
plt.figure()

#------------------------------------------------
# Plot training and validation loss per epoch
#------------------------------------------------
plt.plot  ( epochs,     loss )
plt.plot  ( epochs, val_loss )
plt.title ('Training and validation loss'   )

# 만약 이미지 확인이 필요할 경우 아래 코드로 확인

In [ ]:
# PLOT LOSS AND ACCURACY
%matplotlib inline

import matplotlib.image  as mpimg
import matplotlib.pyplot as plt

#-----------------------------------------------------------
# Retrieve a list of list results on training and test data
# sets for each training epoch
#-----------------------------------------------------------
acc=history.history['acc']
val_acc=history.history['val_acc']
loss=history.history['loss']
val_loss=history.history['val_loss']

epochs=range(len(acc)) # Get number of epochs

#------------------------------------------------
# Plot training and validation accuracy per epoch
#------------------------------------------------
plt.plot(epochs, acc, 'r', "Training Accuracy")
plt.plot(epochs, val_acc, 'b', "Validation Accuracy")
plt.title('Training and validation accuracy')
plt.figure()

#------------------------------------------------
# Plot training and validation loss per epoch
#------------------------------------------------
plt.plot(epochs, loss, 'r', "Training Loss")
plt.plot(epochs, val_loss, 'b', "Validation Loss")


plt.title('Training and validation loss')

# Desired output. Charts with training and validation metrics. No crash :)

### 혹시나 이미지 split이 직접 필요한 경우

In [ ]:
import os
import zipfile
import random
import tensorflow as tf
import shutil
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from shutil import copyfile
from os import getcwd
# Use os.mkdir to create your directories
# You will need a directory for cats-v-dogs, and subdirectories for training
# and testing. These in turn will need subdirectories for 'cats' and 'dogs'
try:
    os.mkdir('/tmp/cats-v-dogs')
    os.mkdir('/tmp/cats-v-dogs/training')
    os.mkdir('/tmp/cats-v-dogs/testing')
    os.mkdir('/tmp/cats-v-dogs/training/cats')
    os.mkdir('/tmp/cats-v-dogs/training/dogs')
    os.mkdir('/tmp/cats-v-dogs/testing/cats')
    os.mkdir('/tmp/cats-v-dogs/testing/dogs')
except OSError:
    print('error')
    pass
# Write a python function called split_data which takes
# a SOURCE directory containing the files
# a TRAINING directory that a portion of the files will be copied to
# a TESTING directory that a portion of the files will be copie to
# a SPLIT SIZE to determine the portion
# The files should also be randomized, so that the training set is a random
# X% of the files, and the test set is the remaining files
# SO, for example, if SOURCE is PetImages/Cat, and SPLIT SIZE is .9
# Then 90% of the images in PetImages/Cat will be copied to the TRAINING dir
# and 10% of the images will be copied to the TESTING dir
# Also -- All images should be checked, and if they have a zero file length,
# they will not be copied over
#
# os.listdir(DIRECTORY) gives you a listing of the contents of that directory
# os.path.getsize(PATH) gives you the size of the file
# copyfile(source, destination) copies a file from source to destination
# random.sample(list, len(list)) shuffles a list
def split_data(SOURCE, TRAINING, TESTING, SPLIT_SIZE):
    # YOUR CODE STARTS HERE
    files = []
    print('original dataset length', len(os.listdir(SOURCE)))
    
    # 3. All images should be checked
    for filename in os.listdir(SOURCE):
        is_file = SOURCE + filename
        
        if os.path.getsize(is_file)>0:
            # bug 발견....
            #files.append(is_file)
            files.append(filename)
        else:
            print(filename + " is zero length!!")
        
    # 1. Train - Test image split
    # 1.1 Get amount of image using portion
    training_list_len = int(len(files) * SPLIT_SIZE)
    testing_list_len = int(len(files) - training_list_len)
    print('split', training_list_len, testing_list_len)
    
    # 2. shuffle하기 -> shuffle한 리스트에서 이미지를 카피할 수 있도록
    shuffled_dataset = random.sample(files, len(files))
    
    # 1.2 Divide training and testing image from shuffeld dataset
    training_dataset = shuffled_dataset[:training_list_len]
    testing_dataset = shuffled_dataset[-testing_list_len:]
    print("shuffle", len(training_dataset), len(testing_dataset))
    
    # 1.3 Copy data to TRAINING and TESTING each (using copyfile)
    for filename in training_dataset:        
        tmp  = SOURCE + filename        
        dst = TRAINING + filename
        copyfile(tmp, dst)
        
    for filename in testing_dataset:
        tmp = SOURCE + filename
        dst = TESTING + filename
        copyfile(tmp, dst)
        
    # YOUR CODE ENDS HERE


CAT_SOURCE_DIR = "/tmp/PetImages/Cat/"
TRAINING_CATS_DIR = "/tmp/cats-v-dogs/training/cats/"
TESTING_CATS_DIR = "/tmp/cats-v-dogs/testing/cats/"
DOG_SOURCE_DIR = "/tmp/PetImages/Dog/"
TRAINING_DOGS_DIR = "/tmp/cats-v-dogs/training/dogs/"
TESTING_DOGS_DIR = "/tmp/cats-v-dogs/testing/dogs/"

split_size = .9
split_data(CAT_SOURCE_DIR, TRAINING_CATS_DIR, TESTING_CATS_DIR, split_size)
split_data(DOG_SOURCE_DIR, TRAINING_DOGS_DIR, TESTING_DOGS_DIR, split_size)